In [1]:
import os
import time
import pickle
import random
import numpy as np
import tensorflow as tf
import sys
from input import DataInput, DataInputTest
from model import Model


/Users/rongyu/opt/anaconda3/envs/tensorflow1.7/lib/python3.5/importlib/_bootstrap.py:222: RuntimeWarning: compiletime version 3.6 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.5
  return f(*args, **kwds)


In [2]:

best_auc = 0.0
def calc_auc(raw_arr):
    """Summary

    Args:
        raw_arr (TYPE): Description

    Returns:
        TYPE: Description
    """
    # sort by pred value, from small to big
    arr = sorted(raw_arr, key=lambda d:d[2])

    auc = 0.0
    fp1, tp1, fp2, tp2 = 0.0, 0.0, 0.0, 0.0
    for record in arr:
        fp2 += record[0] # noclick
        tp2 += record[1] # click
        auc += (fp2 - fp1) * (tp2 + tp1)
        fp1, tp1 = fp2, tp2

    # if all nonclick or click, disgard
    threshold = len(arr) - 1e-3
    if tp2 > threshold or fp2 > threshold:
        return -0.5

    if tp2 * fp2 > 0.0:  # normal auc
        return (1.0 - auc / (2.0 * tp2 * fp2))
    else:
        return None

    
def _auc_arr(score):
    score_p = score[:,0]
    score_n = score[:,1]
    #print "============== p ============="
    #print score_p
    #print "============== n ============="
    #print score_n
    score_arr = []
    for s in score_p.tolist():
        score_arr.append([0, 1, s])
    for s in score_n.tolist():
        score_arr.append([1, 0, s])
    return score_arr


def _eval(model):
    auc_sum = 0.0
    score_arr = []
    for _, uij in DataInputTest(test_set, test_batch_size):
        auc_, score_ = model.eval(uij)
        score_arr += _auc_arr(score_)
        auc_sum += auc_ * len(uij[0])
    test_gauc = auc_sum / len(test_set)
    Auc = calc_auc(score_arr)
    global best_auc
    if best_auc < test_gauc:
        best_auc = test_gauc
        model.save('save_path/ckpt')
    return test_gauc, Auc



# 1. 加载数据（list）

In [3]:
with open('../raw_data/dataset.pkl', 'rb') as f:
    train_set = pickle.load(f)
    test_set = pickle.load(f)
    cate_list = pickle.load(f)
    user_count, item_count, cate_count = pickle.load(f)

In [4]:
train_set[:2]

[(103944, [17704, 37473], 53346, 0), (126219, [15082, 19768, 30450], 48620, 0)]

# 2.模型构建和训练

In [5]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'
random.seed(1234)
np.random.seed(1234)
tf.set_random_seed(1234)

train_batch_size = 32
test_batch_size = 512
epoch_size = round(len(train_set) / train_batch_size)

model = Model(user_count, item_count, cate_count, cate_list)

[None, None, 128]
Instructions for updating:
keep_dims is deprecated, use keepdims instead
[None, 2]


In [6]:
# print('test_gauc: %.4f\t test_auc: %.4f' % _eval(sess, model))
sys.stdout.flush()
lr = 1.0
start_time = time.time()
for _ in range(50):
    random.shuffle(train_set)
    loss_sum = 0.0
    i=0
    for _, uij in DataInput(train_set, train_batch_size):
#         i = i+1
#         if i>10:
#             break
            
        loss = model.train(uij, lr)
        loss_sum += loss
        print('global_step:', model.global_step.eval(), '; loss=', loss, "; loss_sum=", loss_sum)

        if model.global_step.eval() % 1000 == 0:
            test_gauc, Auc = _eval(model)
            print('Epoch %d Global_step %d\tTrain_loss: %.4f\tEval_GAUC: %.4f\tEval_AUC: %.4f' %
              (model.global_epoch_step.eval(), model.global_step.eval(),
               loss_sum / 1000, test_gauc, Auc))
            sys.stdout.flush()
            loss_sum = 0.0

        if model.global_step.eval() % 336000 == 0:
            lr = 0.1

    print('Epoch %d DONE\tCost time: %.2f' %
          (model.global_epoch_step.eval(), time.time()-start_time))
    sys.stdout.flush()
    model.global_epoch_step_op.eval()

print('best test_gauc:', best_auc)
sys.stdout.flush()

global_step: 1 ; loss= 0.75346696 ; loss_sum= 0.7534669637680054
global_step: 2 ; loss= 1.3678739 ; loss_sum= 2.1213408708572388
global_step: 3 ; loss= 1.883981 ; loss_sum= 4.0053218603134155
global_step: 4 ; loss= 0.7643907 ; loss_sum= 4.769712567329407
global_step: 5 ; loss= 0.74844027 ; loss_sum= 5.518152832984924
global_step: 6 ; loss= 0.8749274 ; loss_sum= 6.393080234527588
global_step: 7 ; loss= 0.8490561 ; loss_sum= 7.242136359214783
global_step: 8 ; loss= 0.6926694 ; loss_sum= 7.934805750846863
global_step: 9 ; loss= 0.6878935 ; loss_sum= 8.62269926071167
global_step: 10 ; loss= 1.0701306 ; loss_sum= 9.692829847335815
global_step: 11 ; loss= 0.9381058 ; loss_sum= 10.630935668945312
global_step: 12 ; loss= 0.6926906 ; loss_sum= 11.323626279830933
global_step: 13 ; loss= 0.6993798 ; loss_sum= 12.023006081581116
global_step: 14 ; loss= 0.6767678 ; loss_sum= 12.699773907661438
global_step: 15 ; loss= 0.62171805 ; loss_sum= 13.321491956710815
global_step: 16 ; loss= 0.9155418 ; loss

global_step: 145 ; loss= 0.7047794 ; loss_sum= 104.9199378490448
global_step: 146 ; loss= 0.6873876 ; loss_sum= 105.60732543468475
global_step: 147 ; loss= 0.71045196 ; loss_sum= 106.31777739524841
global_step: 148 ; loss= 0.70631117 ; loss_sum= 107.02408856153488
global_step: 149 ; loss= 0.6903404 ; loss_sum= 107.71442896127701
global_step: 150 ; loss= 0.69421405 ; loss_sum= 108.40864300727844
global_step: 151 ; loss= 0.7147243 ; loss_sum= 109.12336730957031
global_step: 152 ; loss= 0.6835527 ; loss_sum= 109.80691999197006
global_step: 153 ; loss= 0.6833644 ; loss_sum= 110.49028438329697
global_step: 154 ; loss= 0.7187941 ; loss_sum= 111.2090784907341
global_step: 155 ; loss= 0.68143475 ; loss_sum= 111.89051324129105
global_step: 156 ; loss= 0.6954106 ; loss_sum= 112.58592385053635
global_step: 157 ; loss= 0.71352965 ; loss_sum= 113.29945349693298
global_step: 158 ; loss= 0.670908 ; loss_sum= 113.97036147117615
global_step: 159 ; loss= 0.7152351 ; loss_sum= 114.68559658527374
global_s

global_step: 279 ; loss= 0.7192104 ; loss_sum= 197.60787385702133
global_step: 280 ; loss= 0.66194266 ; loss_sum= 198.2698165178299
global_step: 281 ; loss= 0.70046556 ; loss_sum= 198.9702820777893
global_step: 282 ; loss= 0.63148606 ; loss_sum= 199.60176813602448
global_step: 283 ; loss= 0.69646513 ; loss_sum= 200.29823327064514
global_step: 284 ; loss= 0.6803593 ; loss_sum= 200.9785925745964
global_step: 285 ; loss= 0.6732757 ; loss_sum= 201.65186828374863
global_step: 286 ; loss= 0.66439795 ; loss_sum= 202.31626623868942
global_step: 287 ; loss= 0.7137308 ; loss_sum= 203.02999705076218
global_step: 288 ; loss= 0.6985234 ; loss_sum= 203.72852045297623
global_step: 289 ; loss= 0.6825377 ; loss_sum= 204.4110581278801
global_step: 290 ; loss= 0.77211905 ; loss_sum= 205.18317717313766
global_step: 291 ; loss= 0.6702834 ; loss_sum= 205.85346055030823
global_step: 292 ; loss= 0.7021203 ; loss_sum= 206.5555808544159
global_step: 293 ; loss= 0.67240095 ; loss_sum= 207.2279818058014
global_st

global_step: 407 ; loss= 0.72156906 ; loss_sum= 285.4837385416031
global_step: 408 ; loss= 0.7033126 ; loss_sum= 286.1870511174202
global_step: 409 ; loss= 0.67421055 ; loss_sum= 286.8612616658211
global_step: 410 ; loss= 0.6858855 ; loss_sum= 287.54714715480804
global_step: 411 ; loss= 0.67426443 ; loss_sum= 288.2214115858078
global_step: 412 ; loss= 0.7129463 ; loss_sum= 288.934357881546
global_step: 413 ; loss= 0.6688271 ; loss_sum= 289.60318499803543
global_step: 414 ; loss= 0.6858717 ; loss_sum= 290.28905671834946
global_step: 415 ; loss= 0.7435034 ; loss_sum= 291.03256011009216
global_step: 416 ; loss= 0.6747663 ; loss_sum= 291.7073264122009
global_step: 417 ; loss= 0.6900495 ; loss_sum= 292.39737594127655
global_step: 418 ; loss= 0.6384791 ; loss_sum= 293.03585505485535
global_step: 419 ; loss= 0.71214783 ; loss_sum= 293.74800288677216
global_step: 420 ; loss= 0.68991476 ; loss_sum= 294.43791764974594
global_step: 421 ; loss= 0.7260307 ; loss_sum= 295.16394835710526
global_step:

global_step: 548 ; loss= 0.73773086 ; loss_sum= 381.41520953178406
global_step: 549 ; loss= 0.6912208 ; loss_sum= 382.10643035173416
global_step: 550 ; loss= 0.6923068 ; loss_sum= 382.7987371683121
global_step: 551 ; loss= 0.6477572 ; loss_sum= 383.4464943408966
global_step: 552 ; loss= 0.6605984 ; loss_sum= 384.10709273815155
global_step: 553 ; loss= 0.6650778 ; loss_sum= 384.77217054367065
global_step: 554 ; loss= 0.6695626 ; loss_sum= 385.44173312187195
global_step: 555 ; loss= 0.71000165 ; loss_sum= 386.15173476934433
global_step: 556 ; loss= 0.6756456 ; loss_sum= 386.8273803591728
global_step: 557 ; loss= 0.6985212 ; loss_sum= 387.525901556015
global_step: 558 ; loss= 0.65138507 ; loss_sum= 388.17728662490845
global_step: 559 ; loss= 0.62368333 ; loss_sum= 388.80096995830536
global_step: 560 ; loss= 0.7047466 ; loss_sum= 389.5057165622711
global_step: 561 ; loss= 0.6362657 ; loss_sum= 390.1419822573662
global_step: 562 ; loss= 0.6721209 ; loss_sum= 390.8141031861305
global_step: 5

...

global_step: 24535 ; loss= 0.43568498 ; loss_sum= 276.97962337732315
global_step: 24536 ; loss= 0.3792671 ; loss_sum= 277.3588904738426
global_step: 24537 ; loss= 0.6635461 ; loss_sum= 278.0224365592003
global_step: 24538 ; loss= 0.48990482 ; loss_sum= 278.5123413801193
global_step: 24539 ; loss= 0.6470669 ; loss_sum= 279.1594082713127
global_step: 24540 ; loss= 0.6021775 ; loss_sum= 279.7615857720375
global_step: 24541 ; loss= 0.6169542 ; loss_sum= 280.37853997945786
global_step: 24542 ; loss= 0.50685763 ; loss_sum= 280.88539761304855
global_step: 24543 ; loss= 0.45208976 ; loss_sum= 281.3374873697758
global_step: 24544 ; loss= 0.5718904 ; loss_sum= 281.9093777835369
global_step: 24545 ; loss= 0.43465054 ; loss_sum= 282.3440283238888
global_step: 24546 ; loss= 0.60815245 ; loss_sum= 282.9521807730198
global_step: 24547 ; loss= 0.3219579 ; loss_sum= 283.2741386592388
global_step: 24548 ; loss= 0.32781538 ; loss_sum= 283.60195404291153
global_step: 24549 ; loss= 0.70784706 ; loss_sum= 2

global_step: 24663 ; loss= 0.64414155 ; loss_sum= 343.1414070427418
global_step: 24664 ; loss= 0.57468873 ; loss_sum= 343.71609577536583
global_step: 24665 ; loss= 0.43895808 ; loss_sum= 344.15505385398865
global_step: 24666 ; loss= 0.41485125 ; loss_sum= 344.56990510225296
global_step: 24667 ; loss= 0.56721437 ; loss_sum= 345.1371194720268
global_step: 24668 ; loss= 0.385067 ; loss_sum= 345.5221864581108
global_step: 24669 ; loss= 0.42318952 ; loss_sum= 345.9453759789467
global_step: 24670 ; loss= 0.60648775 ; loss_sum= 346.55186372995377
global_step: 24671 ; loss= 0.6094518 ; loss_sum= 347.16131550073624
global_step: 24672 ; loss= 0.5311004 ; loss_sum= 347.69241589307785
global_step: 24673 ; loss= 0.5759594 ; loss_sum= 348.2683752775192
global_step: 24674 ; loss= 0.44679284 ; loss_sum= 348.71516811847687
global_step: 24675 ; loss= 0.39874333 ; loss_sum= 349.1139114499092
global_step: 24676 ; loss= 0.77091354 ; loss_sum= 349.8848249912262
global_step: 24677 ; loss= 0.46631128 ; loss_s

global_step: 24786 ; loss= 0.50387406 ; loss_sum= 406.0880987048149
global_step: 24787 ; loss= 0.45327672 ; loss_sum= 406.5413754284382
global_step: 24788 ; loss= 0.41358942 ; loss_sum= 406.9549648463726
global_step: 24789 ; loss= 0.41083804 ; loss_sum= 407.36580288410187
global_step: 24790 ; loss= 0.34751737 ; loss_sum= 407.71332025527954
global_step: 24791 ; loss= 0.55127 ; loss_sum= 408.2645902633667
global_step: 24792 ; loss= 0.3882841 ; loss_sum= 408.6528743505478
global_step: 24793 ; loss= 0.62078595 ; loss_sum= 409.27366030216217
global_step: 24794 ; loss= 0.6681539 ; loss_sum= 409.94181418418884
global_step: 24795 ; loss= 0.5413978 ; loss_sum= 410.48321199417114
global_step: 24796 ; loss= 0.56213164 ; loss_sum= 411.04534363746643
global_step: 24797 ; loss= 0.44259775 ; loss_sum= 411.4879413843155
global_step: 24798 ; loss= 0.5711475 ; loss_sum= 412.05908888578415
global_step: 24799 ; loss= 0.7349609 ; loss_sum= 412.7940497994423
global_step: 24800 ; loss= 0.55645335 ; loss_sum=

global_step: 24907 ; loss= 0.55158716 ; loss_sum= 469.303709089756
global_step: 24908 ; loss= 0.66485465 ; loss_sum= 469.9685637354851
global_step: 24909 ; loss= 0.5557741 ; loss_sum= 470.52433782815933
global_step: 24910 ; loss= 0.40023053 ; loss_sum= 470.92456835508347
global_step: 24911 ; loss= 0.59415966 ; loss_sum= 471.518728017807
global_step: 24912 ; loss= 0.47492856 ; loss_sum= 471.9936565756798
global_step: 24913 ; loss= 0.40975386 ; loss_sum= 472.4034104347229
global_step: 24914 ; loss= 0.48622084 ; loss_sum= 472.8896312713623
global_step: 24915 ; loss= 0.34493458 ; loss_sum= 473.23456585407257
global_step: 24916 ; loss= 0.69852394 ; loss_sum= 473.9330897927284
global_step: 24917 ; loss= 0.5659732 ; loss_sum= 474.49906301498413
global_step: 24918 ; loss= 0.42085022 ; loss_sum= 474.9199132323265
global_step: 24919 ; loss= 0.4364727 ; loss_sum= 475.3563859462738
global_step: 24920 ; loss= 0.707224 ; loss_sum= 476.063609957695
global_step: 24921 ; loss= 0.48220807 ; loss_sum= 47

KeyboardInterrupt: 

# 评估